In [ ]:
%matplotlib inline
import warnings  
warnings.filterwarnings('ignore')

import os
import json
import glob
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objs as go

from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

# [0] 파일 불러오기
다운샘플링된 point cloud는 아래의 data tree로 구성된 json 파일로 제공됩니다.
```
data
  ├ train
  │    ├ "points" : train point cloud ( 5000, (약 1000 x 3) )
  │    └ "label"  : 0 ~ 9 ( classification ) 
  └ test
        └ "points" : test point cloud ( 1000, (약 1000 x 3) )
```


## MNIST 3D Dataset

MNIST 3D Dataset의 클래스는 10개(0 ~ 9)의 숫자로 구성됩니다.

텀 프로젝트에서는 원본을 downsampling 하여 가공한 약 1000 여 개의 point cloud로 변환하여 제공합니다.

In [ ]:
with open("../input/2021-ml-tp-7/mnist3D_data.json", 'r') as j:
    data = json.load(j)

## (cf) point cloud 시각화

해당 셀에서는 graph_objs 라이브러리를 통해 제공되는 Raw 데이터가 어떻게 구성되어 있는지 3차원으로 시각화하여 확인할 수 있습니다.

여러 방향으로 드래그 하여 여러 시각에서 데이터를 보는 것이 가능합니다.


In [ ]:
# 1720 번째 이미지에 대한 출력
image_number = 1720

digit = data["train"][image_number]["label"]
coordinate = np.array(data["train"][image_number]["points"])

layout = go.Layout(title = "Digit " + str(digit))
plot_data = go.Scatter3d(x = coordinate[:,0], y = coordinate[:,1], z = coordinate[:,2],
                    mode = 'markers', marker = dict(size = 3))

fig = go.Figure(data = [plot_data], layout = layout)
fig.show()

# [1] Data Loader
> 이번 실습문제에서는 3D 데이터를 Voxel 단위로 나눈 후, 이로부터 Feature를 추출하며 이를 기반으로 각 클래스를 분류합니다.
> 클래스는 0부터 9까지 10가지로 분류됩니다.

In [ ]:
from torch.utils.data import Dataset

class MnistDataset(Dataset):
    def __init__(self, data, mode='train'):
        self.data = data[mode]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        xyz = np.array(self.data[idx]['points'], dtype=np.float32)
        return {'points':xyz, 'label':self.data[idx]['label']}

# [2] Data Processing : Voxelization


아래에는 Raw 데이터를 Voxel_Grid에 맞춰 Voxelization을 진행한 후, 이를 기반으로 Feature를 추출하기 위한 함수를 정의하였습니다.

※ 주의사항: Voxelization 시 ( 8, 8, 8 ) 사이즈의 Voxel을 사용합니다.

In [ ]:
# Voxel Grid 생성
# 목표 : point_cloud를 사용자가 지정한 voxel_size 크기의 voxel_grid로 분할한다

def VoxelGrid(points, voxel_size=[1, 1, 1]):

    # Voxel Grid를 구하기 위해 각 축 별 최대, 최소값 계산
    min_xyz  = np.min(points, axis=0) - 0.001
    max_xyz  = np.max(points, axis=0) + 0.001

    diff = max(max_xyz - min_xyz ) - ( max_xyz - min_xyz )
    min_xyz  = min_xyz  - diff / 2
    max_xyz  = max_xyz  + diff / 2

    # voxel_grid
    voxel_grid = []

    # point cloud를 각 축 (x,y,z) 에 대한 voxel grid 로 분할
    for i in range(3):
        # Voxel_size는 정수만 가능
        if type(voxel_size[i]) is not int:
            raise TypeError("voxel_size[{}] must be int".format(i))
        segment = np.linspace( min_xyz[i], max_xyz[i], num=(voxel_size[i] + 1))
        voxel_grid.append(segment)

    return voxel_grid # 생성한 voxel_grid 를 반환

In [ ]:
# Voxelization (복셀화) 및 Voxel_Vector 추출
# 목표: 생성한 voxel_grid에 맞추어 point cloud를 각 grid에 맞춰 Voxelization하고, 이를 통해 voxel_vector 

def VoxelVector(points, voxel_grid, voxel_size=[1, 1, 1]):

    n_voxels = voxel_size[0] * voxel_size[1] * voxel_size[2]
    n_x = voxel_size[0]
    n_y = voxel_size[1]
    n_z = voxel_size[2]

    voxelization = np.zeros((len(points), 4), dtype=int)

    # Voxelization
    voxelization[:,0] = np.searchsorted(voxel_grid[0], points[:,0]) - 1
    voxelization[:,1] = np.searchsorted(voxel_grid[1], points[:,1]) - 1
    voxelization[:,2] = np.searchsorted(voxel_grid[2], points[:,2]) - 1

    voxelization[:,3] = ((voxelization[:,1] * n_x) + voxelization[:,0]) + (voxelization[:,2] * (n_x * n_y)) 

    # Voxelization으로부터 voxel_vector 추출
    vector = np.zeros(n_voxels)
    count = np.bincount(voxelization[:,3])
    vector[:len(count)] = count

    vector = vector.reshape(n_z, n_y, n_x)

    return vector # 추출한 Vector 반환


In [ ]:
# Voxel_Feature 추출
# 목표: 추출한 voxel_vector로부터 classification에 feature로 활용할 수 있는 voxel_feature를 추출한다

def VoxelFeature(dataset):
    
    voxel_vectors = []
    labels = []

    for i, pc in enumerate(tqdm(dataset)):

        # point cloud를 np.array(dtype=np.float32)로 변경
        pointcloud = np.array(pc['points'], dtype = np.float32)   

        voxel_grid = VoxelGrid(pointcloud, voxel_size=[8, 8, 8])
        
        vector = VoxelVector(pointcloud, voxel_grid, voxel_size=[8, 8, 8])

        voxel_vectors.append(vector.reshape(-1) / np.max(vector))
        
        labels.append(pc['label'])

    return np.array(voxel_vectors), np.array(labels)

# [3] Model / Evauation


In [ ]:
# [1] 데이터 불러온 후
mnist_train = MnistDataset(data, mode = "train")
mnist_test = MnistDataset(data, mode = "test")

# [2] 불러온 Raw 데이터를 가공합니다.
x_train, y_train = VoxelFeature(mnist_train)
x_test, _ = VoxelFeature(mnist_test)

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

여기서부터 불러온 x_train, y_train, 그리고 x_test를 바탕으로 모델에 적용해봅시다.

In [ ]:
X = x_train
y = y_train

y

In [ ]:
from sklearn.model_selection import train_test_split #train set과 test set으로 데이터 나누기 위한 함수 호출
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.33, random_state=0, stratify=y)

In [ ]:
# 표준화
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# 표준화
sc = MinMaxScaler()
sc.fit(X_train)
X_train_m = sc.transform(X_train)
X_test_m = sc.transform(X_test)

In [ ]:
from sklearn.svm import SVC
clf = SVC(class_weight='balanced', C = 8, gamma = 0.07, kernel = 'rbf', random_state = 0)
clf.fit(X_train_m, y_train)
y_test_pred = clf.predict(X_test_m)
print(clf.score(X_test_m, y_test))

from sklearn import metrics
print(metrics.classification_report(y_test,y_test_pred))   #classification_report
#X.shape

In [ ]:
X.shape

In [ ]:
clf = SVC(class_weight='balanced', C = 8, gamma = 0.07, kernel = 'rbf', random_state = 0)
# break_ties = True
# decision_function_shape = 'ovo'
clf.fit(X_train_m, y_train)
y_test_pred = clf.predict(X_test_m)
print(clf.score(X_test_m, y_test))

from sklearn import metrics
print(metrics.classification_report(y_test,y_test_pred))   #classification_report
#X.shape

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
param_grid={
    'C':[1e3,5e3,1e4,5e4,1e5],
    'gamma':[0.0001,0.0005,0.001,0.005,0.01,0.1],
    'kernel':['rbf'],
    'random_state':[0]
}
clf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5)   
clf.fit(X_m,y)

In [ ]:
clf.best_params_

In [ ]:
param_grid={
    'C':[100, 500, 1000],
    'gamma':[0.08,0.1, 0.5],
    'kernel':['rbf'],
    'random_state':[0]
}
clf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5)   
clf.fit(X_train_m,y_train)

In [ ]:
clf.best_params_

In [ ]:
param_grid={
    'C':[100, 500, 800],
    'gamma':[0.1, 0.2, 0.3],
    'kernel':['rbf'],
    'random_state':[0]
}
clf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=3)   
clf.fit(X_train_m,y_train)

In [ ]:
clf.best_params_

In [ ]:
param_grid={
    'C':[1, 10, 100, 200],
    'gamma':[0.1, 0.12, 0.15],
    'kernel':['rbf'],
    'random_state':[0]
}
clf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=3)   
clf.fit(X_train_m,y_train)

In [ ]:
clf.best_params_

In [ ]:
param_grid={
    'C':[5, 6, 7],
    'gamma':[0.08, 0.1, 0.12],
    'kernel':['rbf'],
    'random_state':[0]
}
clf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=3)   
clf.fit(X_train_m,y_train)

In [ ]:
clf.best_params_

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid={
    'C':[6, 7, 8],
    'gamma':[0.07, 0.08],
    'kernel':['rbf'],
    'random_state':[0]
}
clf=GridSearchCV(SVC(class_weight='balanced'), param_grid, cv=5)   
clf.fit(X,y)

In [ ]:
clf.best_params_

In [ ]:
from sklearn.svm import SVC
sc = MinMaxScaler()
sc.fit(X)
X_m = sc.transform(X)
x_test_m= sc.transform(x_test)

clf = SVC(class_weight='balanced', C = 8, gamma = 0.07, kernel = 'rbf', random_state = 0) #0.91500
#clf = SVC(class_weight='balanced', C = 6, gamma = 0.08, kernel = 'rbf', random_state = 0) 0.91300
#clf = SVC(class_weight='balanced', C = 11, gamma = 0.05, kernel = 'rbf', random_state = 0) 0.91300
clf.fit(X_m, y)

x_test_pred = clf.predict(x_test)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
submit_csv = pd.read_csv('/kaggle/input/2021-ml-tp-7/sample_submit.csv')
submit_csv.head()
#submit_csv['Label'] = test_std
#submit_csv['Label'] = submit_csv['Label'].astype("int")
#submit_csv.to_csv("./result-rf.csv", mode='w', index=False) 

In [ ]:
submit_csv['Label'] = x_test_pred
submit_csv.head()

In [ ]:
submit_csv.to_csv("./result-svm.csv", mode='w', index=False) 